In [28]:
%pip install wandb -q
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

Dataset

In [29]:
from torchvision import transforms
import torchvision
import torch

# Define transforms for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor()
])

# Datasets
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Dataloaders to feed the data in batches
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1000, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

print('No. of train images: {}'.format(len(trainset)))
print('No. of test images: {}'.format(len(testset)))
print('No. of train batches: {}'.format(len(trainloader)))
print('No. of test batches: {}'.format(len(testloader)))

No. of train images: 60000
No. of test images: 10000
No. of train batches: 60
No. of test batches: 100


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [30]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    # Declaring the Architecture
    def __init__(self):
        super(Network,self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
 
    # Forward Pass
    def forward(self, x):
        x = x.view(x.shape[0],-1)    # Flatten the images
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [31]:
def train(network, epoch, criterion, optimizer, trainloader):
    train_loss = 0.0
    # do it!
    acc_train = 0.0
    for data, labels in trainloader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        # Clear the gradients
        optimizer.zero_grad()
        # Forward Pass
        target = network.forward(data)
        # Find the Loss
        loss = criterion(target, labels)
        # Calculate gradients
        loss.backward()
        # Update Weights
        optimizer.step()
        # Calculate Loss
        train_loss += loss.item()
    train_loss = train_loss / len(trainloader)
    return train_loss, acc_train

In [32]:
def validate(network, epoch, criterion, testloader):
    valid_loss = 0.0
    acc_valid = 0.0
    network.eval()     # Optional when not using Model Specific layer
    for data, labels in testloader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        # Forward Pass
        target = network.forward(data)
        # Find the Loss
        loss = criterion(target,labels)
        # Calculate Loss
        valid_loss += loss.item()
    valid_loss = valid_loss / len(trainloader)
    return valid_loss, acc_train

In [33]:
import time

device = 'cuda'

# WandB Configurations (optional)        
config = {
  "learning_rate": 0.01,
  "epochs": 10
}

# Initialize WandB 
wandb.init(name='test_run', 
           project='test_project',
           notes='This is a test run', 
           tags=['Fashion MNIST', 'Test Run'],
           entity='sadekov_rinat', config = config)

network = Network().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network.parameters(), config["learning_rate"])

# Log the network weight histograms (optional)
wandb.watch(network)

num_epochs = config["epochs"]
start_time = time.time()
for epoch in range(1, num_epochs+1):
    loss_train, acc_train = train(network, epoch, criterion, optimizer, trainloader)
    loss_valid, acc_valid = validate(network, epoch, criterion, testloader)
    print('Epoch: {}  Train Loss: {:.4f}  Train Acc: {:.4f}  Valid Loss: {:.4f}  Valid Acc: {:.4f}'.format(epoch, loss_train, acc_train, loss_valid, acc_valid))
    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train Loss": loss_train,
        "Train Acc": acc_train,
        "Valid Loss": loss_valid,
        "Valid Acc": acc_valid})

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

Epoch: 1  Train Loss: 0.7365  Train Acc: 0.0000  Valid Loss: 0.7705  Valid Acc: 0.0000
Epoch: 2  Train Loss: 0.3983  Train Acc: 0.0000  Valid Loss: 0.6629  Valid Acc: 0.0000
Epoch: 3  Train Loss: 0.3481  Train Acc: 0.0000  Valid Loss: 0.6302  Valid Acc: 0.0000
Epoch: 4  Train Loss: 0.3259  Train Acc: 0.0000  Valid Loss: 0.6355  Valid Acc: 0.0000
Epoch: 5  Train Loss: 0.3051  Train Acc: 0.0000  Valid Loss: 0.6094  Valid Acc: 0.0000
Epoch: 6  Train Loss: 0.2967  Train Acc: 0.0000  Valid Loss: 0.6209  Valid Acc: 0.0000
Epoch: 7  Train Loss: 0.2908  Train Acc: 0.0000  Valid Loss: 0.5878  Valid Acc: 0.0000
Epoch: 8  Train Loss: 0.2716  Train Acc: 0.0000  Valid Loss: 0.5843  Valid Acc: 0.0000
Epoch: 9  Train Loss: 0.2552  Train Acc: 0.0000  Valid Loss: 0.5809  Valid Acc: 0.0000
Epoch: 10  Train Loss: 0.2458  Train Acc: 0.0000  Valid Loss: 0.5735  Valid Acc: 0.0000
Time Elapsed : 65.5566s
